In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains 
import pytube

In [6]:
# open browser
def open_browser():
    chrome_driver = os.environ['CHROMEDRIVER'] # path to chrome driver
    browser = webdriver.Chrome(chrome_driver)
    browser.maximize_window()
    return browser

# download the video at a given url
def download_video(url):
    youtube = pytube.YouTube(url)
    video = youtube.streams.first()
    video.download()

# scrape ijf website
def download_fight_videos():
    browser = open_browser()
    # open ranking list
    browser.get('https://www.ijf.org/wrl?category=all')
    # get judoka <td>
    judokas = browser.find_elements_by_css_selector('.name >a')
    # get judokas' link to profile
    judokas_video_section = [name.get_attribute('href') + '/videos' for name in judokas]
    # get judokas' name
    judokas_name = [jn.text for jn in judokas]
    for judoka_i in range(24, 46):
        name = judokas_name[judoka_i]
        video_section_link = judokas_video_section[judoka_i]
        print(f'----- {judoka_i} - {name} -----')
        try:
            # open video section
            profile = browser.get(video_section_link)
            print('ok video section')
            for trial in range(6):
                try:
                    # get url of fight
                    fight_url = browser.find_element_by_id('widget2').get_attribute('src')
                except:
                    print('Sleeping some more')
                    time.sleep(1)
            # download fight video
            download_video(fight_url)
            print('ok download')
    #             time.sleep(2)
            # handle error such as private videos
        except:
            print(f"error for {name}")
#             time.sleep(2)
            pass
    browser.quit()
    
download_fight_videos()

----- 24 - MSHVIDOBADZE Robert -----
ok video section
Sleeping some more
Sleeping some more
error for MSHVIDOBADZE Robert
----- 25 - CHO Guham -----
ok video section
Sleeping some more
Sleeping some more
ok download
----- 26 - SILVA MORALES Ivan Felipe -----
ok video section
Sleeping some more
Sleeping some more
ok download
----- 27 - MARUYAMA Joshiro -----
ok video section
Sleeping some more
ok download
----- 28 - SHISHIME Ai -----
ok video section
Sleeping some more
Sleeping some more
ok download
----- 29 - TONAKI Funa -----
ok video section
Sleeping some more
Sleeping some more
ok download
----- 30 - SONE Akira -----
ok video section
Sleeping some more
Sleeping some more
ok download
----- 31 - TASOEV Inal -----
ok video section
Sleeping some more
Sleeping some more
error for TASOEV Inal
----- 32 - TASHIRO Miku -----
ok video section
Sleeping some more
Sleeping some more
error for TASHIRO Miku
----- 33 - TAKATO Naohisa -----
ok video section
Sleeping some more
Sleeping some more
ok d

In [ ]:
browser = open_browser()
browser.get('https://www.ijf.org/judoka/16577/videos?currentContestCodeLong=gs_hun2020_m_0090_0066')

# Working on the cell below

Trying to scroll in the video list and click the appropriate video.

In [ ]:
items = browser.find_elements_by_css_selector('.video-list-thumb')
for vid_i in range(36):
    element = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div/div/div[2]/div/div[1]')
    action = ActionChains(browser)
    action.move_to_element(items[vid_i]).click().perform()
    time.sleep(.1)
    browser.execute_script("window.scrollTo(0," + str(120*vid_i) + ")")
    time.sleep(.1)

In [ ]:
items[]